## Data

In [1]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "ăn quả nhớ kẻ trồng cây",
    "có chí thì nên"    
]
data_size = len(corpus)

# Define the max vocabulary size and sequence length
vocab_size = 13
sequence_length = 6

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>", "<sos>"]
                                 )
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'<unk>': 0,
 'cây': 4,
 'kẻ': 6,
 'thì': 10,
 'chí': 3,
 '<pad>': 1,
 'nhớ': 7,
 '<sos>': 2,
 'có': 5,
 'nên': 8,
 'quả': 9,
 'trồng': 11,
 'ăn': 12}

In [3]:
data_x = []
data_y = []
for vector in corpus:
    vector = vector.split()    

    for i in range(len(vector)):
        data_x.append(vector[:i])
        data_y.append(vector[i])

print(data_x)
print(data_y)

[[], ['ăn'], ['ăn', 'quả'], ['ăn', 'quả', 'nhớ'], ['ăn', 'quả', 'nhớ', 'kẻ'], ['ăn', 'quả', 'nhớ', 'kẻ', 'trồng'], [], ['có'], ['có', 'chí'], ['có', 'chí', 'thì']]
['ăn', 'quả', 'nhớ', 'kẻ', 'trồng', 'cây', 'có', 'chí', 'thì', 'nên']


In [4]:
# Tokenize and numericalize your samples
def vectorize(x, y, vocab, sequence_length):
    x = ['<sos>'] + x
    print(x)
    x_ids = [vocab[token] for token in x][:sequence_length]
    x_ids = x_ids + [vocab["<pad>"]] * (sequence_length - len(x))
    
    return x_ids, vocab[y]

# Vectorize the samples
data_x_ids = []
data_y_ids = []
for x, y in zip(data_x, data_y):
    x_ids, y_ids = vectorize(x, y, vocab, sequence_length)
    data_x_ids.append(x_ids)
    data_y_ids.append(y_ids)

['<sos>']
['<sos>', 'ăn']
['<sos>', 'ăn', 'quả']
['<sos>', 'ăn', 'quả', 'nhớ']
['<sos>', 'ăn', 'quả', 'nhớ', 'kẻ']
['<sos>', 'ăn', 'quả', 'nhớ', 'kẻ', 'trồng']
['<sos>']
['<sos>', 'có']
['<sos>', 'có', 'chí']
['<sos>', 'có', 'chí', 'thì']


In [5]:
for x, y in zip(data_x_ids, data_y_ids):
    print(x)
    print(y)
    print()

[2, 1, 1, 1, 1, 1]
12

[2, 12, 1, 1, 1, 1]
9

[2, 12, 9, 1, 1, 1]
7

[2, 12, 9, 7, 1, 1]
6

[2, 12, 9, 7, 6, 1]
11

[2, 12, 9, 7, 6, 11]
4

[2, 1, 1, 1, 1, 1]
5

[2, 5, 1, 1, 1, 1]
3

[2, 5, 3, 1, 1, 1]
10

[2, 5, 3, 10, 1, 1]
8



In [6]:
data_x_ids = torch.tensor(data_x_ids, dtype=torch.long)
print(data_x_ids.shape)

torch.Size([10, 6])


In [7]:
data_y_ids = torch.tensor(data_y_ids, dtype=torch.long)
print(data_y_ids.shape)

torch.Size([10])


## Train with full data

In [8]:
class TG_Model(nn.Module):
    def __init__(self, vocab_size, sequence_length):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 4)
        self.recurrent = nn.RNN(4, 4, batch_first=True)
        self.linear = nn.Linear(sequence_length*4, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x,_ = self.recurrent(x)
        x = nn.Flatten()(x)
        x = self.linear(x)
        return x

model = TG_Model(vocab_size, sequence_length)
print(model)

TG_Model(
  (embedding): Embedding(13, 4)
  (recurrent): RNN(4, 4, batch_first=True)
  (linear): Linear(in_features=24, out_features=13, bias=True)
)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [10]:
# train
for _ in range(35):
    optimizer.zero_grad()
    outputs = model(data_x_ids)
    loss = criterion(outputs, data_y_ids)
    print(loss.item())
    loss.backward()
    optimizer.step()

2.465301990509033
2.1762518882751465
1.9014618396759033
1.655034065246582
1.4398491382598877
1.2611773014068604
1.0969451665878296
0.9445131421089172
0.8114486932754517
0.6931222677230835
0.5856572389602661
0.4951648712158203
0.42266446352005005
0.3633149266242981
0.3160364329814911
0.2801904082298279
0.2517450153827667
0.22911885380744934
0.21228890120983124
0.19883322715759277
0.18787655234336853
0.1789168268442154
0.17208918929100037
0.1668219119310379
0.16237109899520874
0.15880036354064941
0.15572845935821533
0.15341338515281677
0.15148329734802246
0.14997252821922302
0.14869017899036407
0.14756810665130615
0.14664249122142792
0.14577887952327728
0.1451040804386139


In [11]:
outputs = model(data_x_ids)
#print(outputs)
print(torch.argmax(outputs, axis=-1))

tensor([12,  9,  7,  6, 11,  4, 12,  3, 10,  8])


In [12]:
data_y_ids

tensor([12,  9,  7,  6, 11,  4,  5,  3, 10,  8])

## Inference

In [13]:
promt = '<sos> có'
promt = promt.split()
promt_ids = [vocab[token] for token in promt][:sequence_length]
promt_ids = promt_ids + [vocab["<pad>"]] * (sequence_length - len(promt))

print(promt_ids)

[2, 5, 1, 1, 1, 1]


In [14]:
for i in range(sequence_length - len(promt)):
    promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
    outputs = model(promt_tensor)
    next_id = torch.argmax(outputs, axis=-1)

    promt_ids[len(promt)+i] = next_id.item()
    print(promt_ids)

[2, 5, 3, 1, 1, 1]
[2, 5, 3, 10, 1, 1]
[2, 5, 3, 10, 8, 1]
[2, 5, 3, 10, 8, 8]


In [15]:
promt_tensor = torch.tensor(promt_ids, dtype=torch.long).reshape(1, -1)
outputs = model(promt_tensor)
next_id = torch.argmax(outputs, axis=-1)
print(next_id.item())

8


In [16]:
{'<unk>': 0,
 'cây': 4,
 'kẻ': 6,
 'thì': 10,
 'chí': 3,
 '<pad>': 1,
 'nhớ': 7,
 '<sos>': 2,
 'có': 5,
 'nên': 8,
 'quả': 9,
 'trồng': 11,
 'ăn': 12}

{'<unk>': 0,
 'cây': 4,
 'kẻ': 6,
 'thì': 10,
 'chí': 3,
 '<pad>': 1,
 'nhớ': 7,
 '<sos>': 2,
 'có': 5,
 'nên': 8,
 'quả': 9,
 'trồng': 11,
 'ăn': 12}